# WPC 만들기

In [ ]:
import pandas as pd

# 1. 엑셀 파일 경로
file_path = ".xlsx"  # 필요시 경로 수정

# 2. 시트 불러오기
df_work = pd.read_excel(file_path, sheet_name="Work")
df_pub = pd.read_excel(file_path, sheet_name="Publication")
df_col = pd.read_excel(file_path, sheet_name="Collection")
df_edge_pub_w = pd.read_excel(file_path, sheet_name="Edge_(Pub-W)")
df_edge_pub_c = pd.read_excel(file_path, sheet_name="Edge_(Pub-C)")

# 3. 병합용 원본 ID 보존
df_work["work_id_orig"] = df_work["id"]
df_pub["pub_id_orig"] = df_pub["id"]
df_col["col_id_orig"] = df_col["id"]

# 4. 컬럼 prefix 부여
df_work_prefixed = df_work.rename(columns=lambda x: f"work_{x}" if x != "work_id_orig" else x)
df_pub_prefixed = df_pub.rename(columns=lambda x: f"pub_{x}" if x != "pub_id_orig" else x)
df_col_prefixed = df_col.rename(columns=lambda x: f"col_{x}" if x != "col_id_orig" else x)

# 5. 엣지 병합
edge_pub_w = df_edge_pub_w.rename(columns={"source id": "pub_id", "target id": "work_id"})
edge_pub_c = df_edge_pub_c.rename(columns={"source id": "pub_id", "target id": "col_id"})
pub_core = pd.merge(edge_pub_c, edge_pub_w, on="pub_id", how="outer")

# 6. 정보 병합 (ID 기준, prefix 이전 ID 사용)
pub_core = pub_core.merge(df_pub_prefixed, left_on="pub_id", right_on="pub_id_orig", how="left")
pub_core = pub_core.merge(df_work_prefixed, left_on="work_id", right_on="work_id_orig", how="left")
pub_core = pub_core.merge(df_col_prefixed, left_on="col_id", right_on="col_id_orig", how="left")

# 7. 연결된 ID 목록 추출
linked_work_ids = pub_core['work_id_x'].dropna().unique().tolist()
linked_pub_ids = pub_core['pub_id_x'].dropna().unique().tolist()
linked_col_ids = pub_core['col_id_x'].dropna().unique().tolist()
# 8. 단독 항목 추출
solo_work = df_work_prefixed[~df_work_prefixed['work_id_orig'].isin(linked_work_ids)].copy()
solo_pub = df_pub_prefixed[~df_pub_prefixed['pub_id_orig'].isin(linked_pub_ids)].copy()
solo_col = df_col_prefixed[~df_col_prefixed['col_id_orig'].isin(linked_col_ids)].copy()

# 9. 최종 출력 컬럼 정의
final_columns = [
    'col_id_orig', 'col_name',
    'pub_id_orig', 'pub_name',
    'work_id_orig', 'work_korname', 'work_chiname',
    'work_titleExam', 'work_style', 'work_original', 'work_translation', 'work_url'
]

# 10. 누락 컬럼 보완 및 정렬
for df in [solo_work, solo_pub, solo_col]:
    for col in final_columns:
        if col not in df.columns:
            df[col] = None
    df = df[final_columns]

pub_core_final = pub_core[final_columns]

# 11. 전체 병합
full_merged = pd.concat([pub_core_final, solo_work, solo_pub, solo_col], ignore_index=True)

# 12. 저장
full_merged.to_csv("full_merged_with_prefix.csv", index=False)

## EPQA생성
### 1. P-A결합

In [ ]:
import pandas as pd

# === 경로 설정 ===
epqa_path = "EPQA.xlsx"

# === 시트 로드 (컬럼명 정정 반영)
df_person = pd.read_excel(epqa_path, sheet_name="Person").rename(columns={"id": "person_id"})
df_answer = pd.read_excel(epqa_path, sheet_name="Answer").rename(columns={"id": "answer_id"})  # writer는 그대로 사용
df_edge_pea = pd.read_excel(epqa_path, sheet_name="Edge_(Pe-A)").rename(
    columns={"source id": "person_id", "target id": "answer_id"}
).dropna().drop_duplicates()

# === Step 1: PA 병합 (단독 A, P 포함)
# answer 기준 병합
df_pa_left = df_answer.merge(df_edge_pea, on="answer_id", how="left") \
                      .merge(df_person, on="person_id", how="left")

# person 기준 병합
df_pa_right = df_person.merge(df_edge_pea, on="person_id", how="left") \
                       .merge(df_answer, on="answer_id", how="left")

# 병합 결과 통합 + 중복 제거
df_pa_all = pd.concat([df_pa_left, df_pa_right], ignore_index=True).drop_duplicates(subset=["person_id", "answer_id"])

# === 작성자 일치 여부 플래그 추가 (writer vs fullname)
def check_writer_match(row):
    try:
        return pd.notna(row["writer"]) and pd.notna(row["fullname"]) and row["writer"].strip() in row["fullname"]
    except:
        return False

df_pa_all["is_writer_match"] = df_pa_all.apply(check_writer_match, axis=1)

# === 저장: 전체, 일치, 불일치 시트 분리
with pd.ExcelWriter("step1_PA_flagged.xlsx") as writer:
    df_pa_all.to_excel(writer, index=False, sheet_name="PA_ALL")
    df_pa_all[df_pa_all["is_writer_match"] == True].to_excel(writer, index=False, sheet_name="PA_Match")
    df_pa_all[df_pa_all["is_writer_match"] == False].to_excel(writer, index=False, sheet_name="PA_Mismatch")

print("✅ Step 1 (PA) 저장 완료: 전체 {}, 일치 {}, 불일치 {}".format(
    len(df_pa_all),
    df_pa_all["is_writer_match"].sum(),
    len(df_pa_all) - df_pa_all["is_writer_match"].sum()
))

### 2. (P-A)Q결합

In [3]:
import pandas as pd

# === 1. 파일 경로
pea_path = "EPQA_PeA_with_edge_hit.xlsx"
epqa2_path = "EPQA2.xlsx"

# === 2. Pe-A 결합 결과 불러오기 (edge_hit1 포함)
df_pea = pd.read_excel(pea_path)
df_pea = df_pea.rename(columns={"edge_hit": "edge_hit1"})

# === 3. A-Q 엣지 및 질문 시트 불러오기 (name 기준)
df_edge_aq = pd.read_excel(epqa2_path, sheet_name="Edge_(A-Q)").rename(
    columns={"source name": "answer_name", "target name": "question_name"}
).dropna().drop_duplicates()

df_question = pd.read_excel(epqa2_path, sheet_name="Question").rename(
    columns={"name": "question_name", "id": "question_id"}
)

# === 4. 병합: A-Q 연결 정보 붙이기
df_paq = df_pea.merge(df_edge_aq, how="left", on="answer_name") \
               .merge(df_question, how="left", on="question_name")

# === 5. edge_hit2 추가: 정밀 매칭 여부 판단
df_edge_aq["key"] = df_edge_aq["answer_name"].astype(str) + "|" + df_edge_aq["question_name"].astype(str)
df_paq["key"] = df_paq["answer_name"].astype(str) + "|" + df_paq["question_name"].astype(str)
df_paq["edge_hit2"] = df_paq["key"].isin(df_edge_aq["key"])
df_paq = df_paq.drop(columns=["key"])

# === 6. 저장
df_paq.to_excel("step2_PeAQ_with_edge_hit2.xlsx", index=False)

print("✅ 완료: 전체 {}, A-Q 연결 수 (edge_hit2=True): {}".format(
    len(df_paq),
    df_paq["edge_hit2"].sum()
))

✅ 완료: 전체 498, A-Q 연결 수 (edge_hit2=True): 346


### 3. (P-A-Q)E 결합

In [4]:
import pandas as pd

# === 1. 경로 설정
step2_path = "step2_PeAQ_with_edge_hit2.xlsx"
epqa2_path = "EPQA2.xlsx"

# === 2. 이전 병합 파일 불러오기 (edge_hit1, edge_hit2 포함)
df_paq = pd.read_excel(step2_path)

# === 3. Q-E 엣지 및 Exam 시트 불러오기
df_edge_qe = pd.read_excel(epqa2_path, sheet_name="Edge_(Q-E)").rename(
    columns={"source name": "question_name", "target name": "exam_name"}
).dropna().drop_duplicates()

df_exam = pd.read_excel(epqa2_path, sheet_name="Exam").rename(
    columns={"name": "exam_name", "id": "exam_id"}
)

# === 4. 병합: Q-E 연결 및 Exam 정보
df_paqe = df_paq.merge(df_edge_qe, how="left", on="question_name") \
                .merge(df_exam, how="left", on="exam_name")

# === 5. edge_hit3 계산 (정확히 연결된 Q-E 쌍에 대해서)
df_edge_qe["key"] = df_edge_qe["question_name"].astype(str) + "|" + df_edge_qe["exam_name"].astype(str)
df_paqe["key"] = df_paqe["question_name"].astype(str) + "|" + df_paqe["exam_name"].astype(str)
df_paqe["edge_hit3"] = df_paqe["key"].isin(df_edge_qe["key"])
df_paqe = df_paqe.drop(columns=["key"])

# === 6. 저장
df_paqe.to_excel("step3_PeAQE_with_edge_hit3.xlsx", index=False)

print("✅ 완료: 전체 {}, Q-E 연결 수 (edge_hit3=True): {}".format(
    len(df_paqe),
    df_paqe["edge_hit3"].sum()
))

✅ 완료: 전체 1734, Q-E 연결 수 (edge_hit3=True): 1428


### 4. CPW-EPQA 연결

In [3]:
import pandas as pd

# === 파일 경로
step3_path = "step3_PeAQE_with_edge_hit3.xlsx"
cpw_path = "CPW.xlsx"

# === 1. step3 파일 로드
df_peaqe = pd.read_excel(step3_path, sheet_name=0)
print("✅ df_peaqe loaded. Columns:", df_peaqe.columns.tolist())

# === 2. A-W 엣지 시트 로드
df_edge_aw = pd.read_excel(cpw_path, sheet_name="Edge_(A-W)").rename(
    columns={
        "source id": "answer_id", 
        "source name": "answer_name", 
        "target id": "work_id", 
        "target name": "work_name"
    }
).dropna().drop_duplicates()
print("✅ df_edge_aw loaded. Columns:", df_edge_aw.columns.tolist())

# === 3. Work 시트 로드
df_work = pd.read_excel(cpw_path, sheet_name="Sheet2").rename(
    columns={"work_id_orig": "work_id"}
)
print("✅ df_work loaded. Columns:", df_work.columns.tolist())

✅ df_peaqe loaded. Columns: ['person_name', 'person_fullname', 'person_korname', 'person_chiname', 'person_middleName', 'person_courtesyName', 'person_alias', 'person_birthYear', 'person_deathYear', 'person_origin', 'person_clan', 'person_residence', 'answer_name', 'answer_writer', 'answer_year', 'answer_sort', 'answer_contents', 'edge_hit1', 'source id_x', 'target id_x', 'question_name', 'question_id', 'category', 'category2', 'abstract', 'contents', 'source', 'source URL', 'edge_hit2', 'source id_y', 'target id_y', 'exam_name', 'exam_id', 'sortA', 'sortB', 'sortC', 'sortD', 'sortE', '정보 출처', 'year', 'edge_hit3']
✅ df_edge_aw loaded. Columns: ['answer_id', 'answer_name', 'work_id', 'work_name']
✅ df_work loaded. Columns: ['col_id_orig', 'col_name', 'pub_id_orig', 'pub_name', 'work_id', 'work_korname', 'work_chiname', 'work_titleExam', 'work_style', 'work_original', 'work_translation', 'work_url', 'work_id', 'pub_id', 'pub_period', 'pub_type', 'pub_institution', 'pub_provider', 'col_id

In [ ]:
import pandas as pd

# === 파일 로드
step3_path = "step3_PeAQE_with_edge_hit3.xlsx"
cpw_path = "CPW.xlsx"

df_peaqe = pd.read_excel(step3_path, sheet_name="Sheet1")
df_edge_aw = pd.read_excel(cpw_path, sheet_name="Edge_(A-W)").rename(
    columns={"source id": "answer_id", "source name": "answer_name", "target id": "work_id"}
)
df_work_raw = pd.read_excel(cpw_path, sheet_name="Sheet2")

# work 데이터 정리
if "work_id" not in df_work_raw.columns:
    df_work_raw = df_work_raw.rename(columns={"work_id_orig": "work_id"})
df_work = df_work_raw.loc[:, ~df_work_raw.columns.duplicated()].copy()
df_work = df_work.drop_duplicates(subset=["work_id"], keep="first")

# 병합 키 생성
df_peaqe["answer_id"] = df_peaqe["source id_x"]
df_peaqe["merge_key"] = df_peaqe["answer_id"].astype(str) + "|" + df_peaqe["answer_name"].astype(str)
df_edge_aw["merge_key"] = df_edge_aw["answer_id"].astype(str) + "|" + df_edge_aw["answer_name"].astype(str)

print(f"Edge_(A-W) 개수: {len(df_edge_aw)}개")
print(f"step3 개수: {len(df_peaqe)}개")

# === 파일 1: Edge 기준으로 매핑 (정확히 291개)
# Edge 각 행에 대해 step3 정보 추가 (left join)
mapped_from_edge = df_edge_aw.merge(df_peaqe, how="left", on="merge_key")
# work 정보 추가
mapped_complete = mapped_from_edge.merge(df_work, how="left", on="work_id")
mapped_complete["edge_hit4"] = True

print(f"매핑 파일: {len(mapped_complete)}개 (Edge와 동일해야 함)")

# === 파일 2: step3에만 있고 Edge에 없는 데이터
step3_only = df_peaqe[~df_peaqe["merge_key"].isin(df_edge_aw["merge_key"])]
print(f"step3 단독: {len(step3_only)}개")

# === 파일 3: CPW work에만 있고 Edge에 연결안된 데이터
connected_work_ids = df_edge_aw["work_id"].unique()
cpw_work_only = df_work[~df_work["work_id"].isin(connected_work_ids)]
print(f"CPW work 단독: {len(cpw_work_only)}개")

# 저장
mapped_complete.to_excel("mapped_edge_291.xlsx", index=False)
step3_only.to_excel("step3_only_unmapped.xlsx", index=False)
cpw_work_only.to_excel("cpw_work_only_unmapped.xlsx", index=False)

print(f"\n✅ 3개 파일 저장:")
print(f"1. mapped_edge_291.xlsx: {len(mapped_complete)}개 (Edge 기준 매핑)")
print(f"2. step3_only_unmapped.xlsx: {len(step3_only)}개 (step3 단독)")
print(f"3. cpw_work_only_unmapped.xlsx: {len(cpw_work_only)}개 (CPW work 단독)")
print(f"📊 총합: {len(mapped_complete) + len(step3_only) + len(cpw_work_only)}개")

# 검증
print(f"\n🔍 검증:")
print(f"Edge_(A-W) 원본: {len(df_edge_aw)}개")
print(f"매핑 파일: {len(mapped_complete)}개") 
print(f"일치 여부: {'✅ 완벽!' if len(mapped_complete) == len(df_edge_aw) else '❌'}")

# step3 정보가 있는 행과 없는 행 확인
step3_matched = mapped_complete[mapped_complete['person_name'].notna()]
step3_unmatched = mapped_complete[mapped_complete['person_name'].isna()]
print(f"매핑파일 중 step3 정보 있음: {len(step3_matched)}개")
print(f"매핑파일 중 step3 정보 없음: {len(step3_unmatched)}개")

## flat 열 핸드크래프트로 정리, 복붙열 정리

In [ ]:
import pandas as pd
import re

# 엑셀 파일 불러오기
df = pd.read_excel('EPQA_flat_revised.xlsx')  # 파일명 변경

# 정규식 패턴: 문자 1 + 숫자 3 + _ + 숫자 3 + 문자 1
pattern = r'[A-Za-z]\d{3}_\d{3}[A-Za-z]'

# Q_abstract 열에서 해당 패턴을 제거
df['Q_abstract'] = df['Q_abstract'].astype(str).apply(lambda x: re.sub(pattern, '', x))

# 결과 저장 (선택사항)
df.to_excel('EPQA_cleaned.xlsx', index=False)

In [ ]:
import pandas as pd

# 데이터 불러오기
df = pd.read_excel('EPQA_cleaned.xlsx')  # 실제 파일명으로 변경
duplicated_rows = df[df.duplicated()]
print(f"중복 행 개수: {len(duplicated_rows)}")

## EPQA flattening
- person-answer 병합

In [21]:
import pandas as pd

# 1. 파일 로딩
file_path = "EPQA2.xlsx"
df_person = pd.read_excel(file_path, sheet_name="Person")
df_answer = pd.read_excel(file_path, sheet_name="Answer")
df_edge_pea = pd.read_excel(file_path, sheet_name="Edge_(Pe-A)")

# 2. 컬럼명 정리
df_edge_pea = df_edge_pea.rename(columns={
    "source id": "edge_person_id",
    "target id": "edge_answer_id",
    "source name": "person_name",
    "target name": "answer_name"
})
df_person = df_person.rename(columns={
    "fullname": "person_fullname",
    "korname": "person_korname",
    "chiname": "person_chiname",
    "middleName": "person_middleName",
    "courtesyName": "person_courtesyName",
    "alias": "person_alias",
    "birthYear": "person_birthYear",
    "deathYear": "person_deathYear",
    "origin": "person_origin",
    "clan": "person_clan",
    "Residence(방목)": "person_residence"
})
df_answer = df_answer.rename(columns={
    "name": "answer_name",
    "writer": "answer_writer",
    "year": "answer_year",
    "sort": "answer_sort",
    "contents": "answer_contents"
})

# 3. 병합: edge 기반 병합 (edge_hit=True)
df_edge_merged = df_edge_pea \
    .merge(df_person, how="left", left_on="person_name", right_on="person_fullname") \
    .merge(df_answer, how="left", on="answer_name")
df_edge_merged["edge_hit"] = True  # 이건 확실히 edge에서 연결된 것

# 4. 병합: person-answer 전체 조합 (edge와 관계없이)
df_all = df_person.merge(df_answer, how="outer", left_on="person_fullname", right_on="answer_writer")

# 5. edge에 해당하는 것만 True로, 나머지는 False
df_all["key"] = df_all["person_fullname"].astype(str) + "|" + df_all["answer_name"].astype(str)
df_edge_merged["key"] = df_edge_merged["person_fullname"].astype(str) + "|" + df_edge_merged["answer_name"].astype(str)

df_all["edge_hit"] = df_all["key"].isin(df_edge_merged["key"])
df_all = df_all.drop(columns=["key"])

# 🔧 중복 제거 후 안전한 매핑
person_name_map = df_edge_pea[["answer_name", "person_name"]].drop_duplicates(subset="answer_name") \
                                                             .set_index("answer_name")["person_name"]
df_all["person_name"] = df_all["answer_name"].map(person_name_map)
# 6. 최종 선택 컬럼 (안전하게 확인)
columns_to_keep = [col for col in [
    "person_name", "person_fullname", "person_korname", "person_chiname",
    "person_middleName", "person_courtesyName", "person_alias",
    "person_birthYear", "person_deathYear", "person_origin", "person_clan", "person_residence",
    "answer_name", "answer_writer", "answer_year", "answer_sort", "answer_contents",
    "edge_hit"
] if col in df_all.columns]

df_result = df_all[columns_to_keep]
df_result.to_excel("EPQA_PeA_with_edge_hit.xlsx", index=False)

In [10]:
df_mismatch = df_complete[df_complete['person_name'] != df_complete['person_fullname']]
print("❗ person_name ≠ person_fullname")
print(df_mismatch[['person_name', 'person_fullname']].drop_duplicates())

❗ person_name ≠ person_fullname
      person_name person_fullname
0             NaN             NaN
30848    황성한(黃聖漢)             NaN


In [11]:
df_mismatch = df_complete[df_complete['person_name'] != df_complete['answer_writer']]
print("❗ person_name ≠ answer_writer")
print(df_mismatch[['person_name', 'answer_writer', 'answer_name']].drop_duplicates())

❗ person_name ≠ answer_writer
      person_name answer_writer        answer_name
0             NaN        김담(金淡)  1446년_중시_문과_■시_답안
20            NaN      성삼문(成三問)  1446년_중시_문과_■시_답안
40         김담(金淡)           NaN  1446년_중시_문과_■시_문제
48       성삼문(成三問)           NaN  1446년_중시_문과_■시_문제
56            NaN        김흔(金訢)  1471년_별시_문과_■시_답안
...           ...           ...                ...
32862      한충(韓忠)      이건명(李健命)                NaN
32902      한충(韓忠)      손명래(孫命來)                NaN
32942      한충(韓忠)        이만(李槾)                NaN
33022      한충(韓忠)      최창대(崔昌大)                NaN
33062      한충(韓忠)      신정하(申靖夏)                NaN

[119 rows x 3 columns]


In [12]:
df_edge_pea[['person_name', 'answer_name']].drop_duplicates()

,person_name,answer_name
0,윤대순(尹大淳),1813년_증광시_문과_복시_종장_답안
1,나세찬(羅世纘),1527년_정시_문과_초시_답안
2,오상(吳祥),1531년_식년시_진사시_복시_초장_답안
3,정철(鄭澈),1562년_별시_문과_전시_답안
4,홍성민(洪聖民),1564년_식년시_문과_전시_답안
...,...,...
256,고부천(高傅川),곽자의궁사극치지론(郭子儀窮奢極侈之論)
257,안헌징(安獻徵),용흥치운이십운(龍興致雲二十韻)
258,윤선도(尹善道),잠화일지론(簪花一枝論)
259,홍서봉(洪瑞鳳),의당곽자의사봉분양왕표(擬唐郭子儀謝封汾陽王表)
